In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(temperature=0.1)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=60,
)

doc = loader.load_and_split(text_splitter=splitter)
cache_dir = LocalFileStore("./.cache/")

embedder = OpenAIEmbeddings()
cache_embedder = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

vectorStore = Chroma.from_documents(doc, cache_embedder)

retriver = vectorStore.as_retriever()

map_docs_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the 
            text is relevant to answer the question. Return any relevant text
            verbatim.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_docs_chain = map_docs_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    results = []
    for doc in documents:
        result = map_docs_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        results.append(result)
    results = "\n\n".join(results)
    print(ResourceWarning)
    return results
    """
    return "\n\n".join(
        map_docs_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        for doc in documents
    )
    """


map_chain = {"documents": retriver, "question": RunnablePassthrough()} | RunnableLambda(
    map_docs
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a
            question, create a final answer.
            If you don't know the answer, just say that you don't know. Don't try
            to make up an answer
            ------
            {context}
            """,
        ),
        ("human", "question"),
    ]
)

final_Chain = (
    {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
)

final_Chain.invoke("Describe Victory Mansions")

AIMessage(content="I don't know the answer to your question.")